In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Define the path to the two folders
#folder_1_path = '/content/drive/MyDrive/Dataset/byteplot/malicious'
#folder_2_path = '/content/drive/MyDrive/Dataset/byteplot/benign/benign'
#folder_3_path = '/content/drive/MyDrive/Dataset/byteplot/benign/benign_edu'
new_folder_1_path = '/content/drive/MyDrive/Dataset/malicious_1000'
new_folder_2_path = '/content/drive/MyDrive/Dataset/benign_1000'

# Read the images in each folder and store them in a list
images_1 = [cv2.imread(os.path.join(new_folder_1_path, image_file), cv2.IMREAD_GRAYSCALE) \
            for image_file in os.listdir(new_folder_1_path)]
images_2 = [cv2.imread(os.path.join(new_folder_2_path, image_file), cv2.IMREAD_GRAYSCALE) \
            for image_file in os.listdir(new_folder_2_path)]
#images_3 = [cv2.imread(os.path.join(folder_3_path, image_file), cv2.IMREAD_GRAYSCALE) \
 #           for image_file in os.listdir(folder_3_path)]

# Label each image with the respective label (1 for total_des_trainfolder_1, 2 for folder_2)
labels_1 = [0 for _ in range(len(images_1))]
labels_2 = [1 for _ in range (len(images_2))]
#labels_2 = [1 for _ in range(len(images_2)+len(images_3))]

# Combine the images and labels into a single dataset
images = images_1 + images_2 #+ images_3
labels = labels_1 + labels_2

X = images
y = labels

In [6]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from google.colab.patches import cv2_imshow
import numpy as np
from PIL import Image

def gabor_filter(ksize, sigma, theta, lambd, gamma, psi):
    filter_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, cv2.CV_32F)
    return filter_kernel

def apply_gabor_filter(img, filter_kernel):
    filtered_img = cv2.filter2D(img, cv2.CV_8UC3, filter_kernel)
    return filtered_img

def extract_features(filtered_img):
    mean = np.mean(filtered_img)
    var = np.var(filtered_img)
    feature_vector = np.concatenate((np.array([mean]), np.array([var])))
    return feature_vector

def Gabor_filter_extraction(img):
    # Define the parameters for the 24 Gabor filters
    ksize = 5
    sigma_list = [1, 2, 3]
    theta_list = [0, np.pi/8, np.pi/4, 3*np.pi/8]
    lambd_list = [0.05, 0.25]
    gamma = 0.5
    psi = 0.0

    filtered_img_list = []
    for sigma in sigma_list:
        for theta in theta_list:
            for lambd in lambd_list:
                # Create the filter kernel
                filter_kernel = gabor_filter(ksize, sigma, theta, lambd, gamma, psi)
                # Apply the filter to the image
                filtered_img = apply_gabor_filter(img, filter_kernel)
                filtered_img_list.append(filtered_img)
    feature_vector = []
    for i, filtered_img in enumerate(filtered_img_list):
      filtered_img = cv2.normalize(filtered_img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
      features = extract_features(filtered_img)
      feature_vector.append(features)

    hist = cv2.calcHist([img], [0], None, [16], [0, 256])
    feature_vector.append(hist)
    return feature_vector


In [54]:
total_des_test = [Gabor_filter_extraction(img) for img in X_test] 

In [8]:
total_des_train = [Gabor_filter_extraction(img) for img in X_train] 

In [48]:
# reshape the filters
tot_train_features = []

for i in total_des_train:
  x = np.hstack(i[:24])
  z = i[-1].flatten()
  w = np.concatenate((x,z))
  tot_train_features.append(w)

In [60]:
tot_test_features = []

for i in total_des_test:
  x = np.hstack(i[:24])
  z = i[-1].flatten()
  w = np.concatenate((x,z))
  tot_test_features.append(w)

In [61]:
tot_test_features = np.array(tot_test_features)

In [49]:
tot_train_features = np.array(tot_train_features)

In [62]:
tot_test_features.shape

(400, 64)

In [69]:
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(tot_train_features, y_train)

score = clf.score(tot_test_features, y_test)
print("Test accuracy:", score)

Test accuracy: 0.8975
